In [2]:
import sys
sys.path.append('C:/Users/pc/Documents/GitHub/Private_Project/personal_project')
from collections import defaultdict
from datetime import datetime, timedelta
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pickle
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
from functools import partial
# [Default]

def OS_Driver(os,browser):
    if os.lower() == 'windows':
        if browser.lower() == 'phantom':
            driver = webdriver.PhantomJS('C:/Users/pc/Documents/phantomjs-2.1.1-window/bin/phantomjs.exe')
        else:
            driver = webdriver.Chrome('C:/Users/pc/Documents/chromedriver.exe')
    elif os.lower() == 'mac':
        if browser.lowser() == 'phantom':
            driver = webdriver.PhantomJS(
                '/Users/hyunyoun/Documents/GitHub/Private_Project/phantomjs-2.1.1/bin/phantomjs')
        else:
            driver = webdriver.Chrome('../chromedriver')
    return driver
def OpenAPI(apiFile):
    apiKey = pickle.load(open(apiFile, 'rb'))
    return apiKey
## Define Error
class NotMatch(Exception):
    def __init__(self, f):
        self.f = f
    def __str__(self):
        return self.f
## Setting BeautifulSoup
def Setting_BeautifulSoup(parser):
    if parser.lower() == 'xml':
        parser = 'lxml'
    elif parser.lower() == 'html':
        parser = 'html.parser'
    else:
        raise NotMatch('Not Match parser')
    return parser
## Using BeautifulSoup
def WebPageUsingBS(url, htmltype, openApi=None):
    htmltype = Setting_BeautifulSoup(htmltype)
    if openApi == None:
        response = requests.get(url)
    else:
        response = requests.get(url, headers=openApi)
    soup = BeautifulSoup(response.content, htmltype)
    return soup

In [3]:
driver = webdriver.Chrome('../../../chromedriver.exe')

In [4]:
w2 = 'http://v.media.daum.net/v/20171201163917957'

In [5]:
driver.get(w2)

In [6]:
try:
    element = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'alex-area')))
except:pass
else:
    loop = True
    while loop:
        try:
            commentsByclass = 'alex_more'
            element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CLASS_NAME, commentsByclass)))
            more_button = driver.find_element_by_class_name(commentsByclass)
            webdriver.ActionChains(driver).move_to_element(more_button).click(more_button).perform()
            commentElements = driver.find_elements_by_class_name('cmt_info')
        except:
            loop = False

In [9]:
def CommentsInDaum(df):
    df[r'공감'] = re.search('[\d]+', df[r'공감']).group()
    df[r'비공감'] = re.search('[\d]+', df[r'비공감']).group()
    if df['comments'] == '':
        comments = '오직 이모티콘만 있는 댓글입니다.'
    else:
        comments = ' '.join(df['comments'].split('\n'))
    df['comments'] = comments
    return df

In [10]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
comments= soup.select('#alex-area > div > div > div > div.cmt_box > ul.list_comment > li > div.cmt_info')

In [11]:
xx = list(map(lambda x: (x.find('p', class_ = 'desc_txt').text, x.find('button',class_ = '#like').text, x.find('button',class_ = '#dislike').text), comments))
xx = pd.DataFrame(xx)

AttributeError: 'NoneType' object has no attribute 'text'

In [17]:
for i in comments:
    try:
        i.find('p',class_='desc_txt').text
        i.find('button',class_ = '#like').text
        i.find('button',class_ = '#dislike').text
    except:
        print (i)
        break

<div class="cmt_info" data-reactid=".0.0.0.4.1.$194178383.0"><strong class="tit_nick" data-reactid=".0.0.0.4.1.$194178383.0.0"><span class="info_author" data-reactid=".0.0.0.4.1.$194178383.0.0.0"><span class="link_nick" data-reactid=".0.0.0.4.1.$194178383.0.0.0.0"><span class="img_daum" data-reactid=".0.0.0.4.1.$194178383.0.0.0.0.0"></span></span><a class="link_nick" data-reactid=".0.0.0.4.1.$194178383.0.0.0.1">탈퇴한 사용자</a><span class="txt_date" data-reactid=".0.0.0.4.1.$194178383.0.0.0.2">2017.12.01 21:11</span></span></strong><p class="desc_txt font_size_17" data-reactid=".0.0.0.4.1.$194178383.0.2"><em class="emph_txt" data-reactid=".0.0.0.4.1.$194178383.0.2.0">삭제</em><span data-reactid=".0.0.0.4.1.$194178383.0.2.1">된 글입니다.</span></p></div>


In [18]:
i

<div class="cmt_info" data-reactid=".0.0.0.4.1.$194178383.0"><strong class="tit_nick" data-reactid=".0.0.0.4.1.$194178383.0.0"><span class="info_author" data-reactid=".0.0.0.4.1.$194178383.0.0.0"><span class="link_nick" data-reactid=".0.0.0.4.1.$194178383.0.0.0.0"><span class="img_daum" data-reactid=".0.0.0.4.1.$194178383.0.0.0.0.0"></span></span><a class="link_nick" data-reactid=".0.0.0.4.1.$194178383.0.0.0.1">탈퇴한 사용자</a><span class="txt_date" data-reactid=".0.0.0.4.1.$194178383.0.0.0.2">2017.12.01 21:11</span></span></strong><p class="desc_txt font_size_17" data-reactid=".0.0.0.4.1.$194178383.0.2"><em class="emph_txt" data-reactid=".0.0.0.4.1.$194178383.0.2.0">삭제</em><span data-reactid=".0.0.0.4.1.$194178383.0.2.1">된 글입니다.</span></p></div>

In [26]:
comments = list(filter(lambda x: x.select_one('p.desc_txt > em.emph_txt') == None, comments))
comments = list(map(lambda x: (x.find('p', class_ = 'desc_txt').text, x.find('button',class_ = '#like').text, x.find('button',class_ = '#dislike').text), comments))

In [27]:
comments

[('정신과의사란 사람이 저게 뭐냐..무식하긴..', '댓글 찬성하기13506', '댓글 비추천하기1051'),
 ('유아인이  어때서 그러지? 소신있고 좋은데..ㅉㅉ', '댓글 찬성하기13211', '댓글 비추천하기2532'),
 ('다들시끄럽다...', '댓글 찬성하기7483', '댓글 비추천하기459'),
 ('결국 일은 가만히 있는 유아인을 건들여 놓고...결국 정신병자까지 몰고 간거 아녀.....\n참 오지랖도 많은 것들...',
  '댓글 찬성하기5523',
  '댓글 비추천하기954'),
 ('애호박으로 시비걸어서\n애호박으로 받아친게 뭐가 문제라고', '댓글 찬성하기5201', '댓글 비추천하기703'),
 ('sns는 인생의 낭비.... 가 진리...', '댓글 찬성하기3893', '댓글 비추천하기126'),
 ('뭐가 어쨌든간에\n가만히 있는 유아인 걸고 넘어지면서\n생긴 일 아닌가?\n애호박이 코를 찡끗하든말든\n지들 인생이나 신경들 쓰지\n뭔 자기 일 잘하고 사는 멀쩡한\n사람을 걸고 넘어지는거야?',
  '댓글 찬성하기4048',
  '댓글 비추천하기637'),
 ('저런게 의사라니. 누가 진료 받나?', '댓글 찬성하기3661', '댓글 비추천하기513'),
 ('의사라는 인간이 전국구로 유아인이 조증증세 있다고 공개하는거야 뭐야\n저런게 의사라니..',
  '댓글 찬성하기3430',
  '댓글 비추천하기372'),
 ('"애호박으로 맞아 봤음"   이   왜   문제가   되는건가요?????? \n\n진짜   잘   이해가  안됨 ㅠㅠ',
  '댓글 찬성하기2937',
  '댓글 비추천하기546'),
 ('의사들이 우라지게 할일없네\n니네들 정신병부터 치료해', '댓글 찬성하기2384', '댓글 비추천하기251'),
 ('sns는 인생의 낭비 -퍼거슨', '댓글 찬성하기1800', '댓글 비추천하기82'),
 ('정신과 의사 저 시끼도 정신병 의증임.. 방송좀 탔다고 자신이 유명인인줄 알고 뭐해도 된다고 생각하는병.... 경리플리증후

2233